<a href="https://colab.research.google.com/github/PP002/Multi-Cloud-Transfer-Tool/blob/main/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [1]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libtorrent-rasterbar2.0
The following NEW packages will be installed:
  libtorrent-rasterbar2.0 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,098 kB of archives.
After this operation, 7,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtorrent-rasterbar2.0 amd64 2.0.5-5 [1,501 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-libtorrent amd64 2.0.5-5 [597 kB]
Fetched 2,098 kB in 1s (1,511 kB/s)
Selecting previously unselected package libtorrent-rasterbar2.0:amd64.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../libtorrent-rasterbar2.0_2.0.5-5_amd64.deb ...
Unpacking libtorrent-rasterbar2.0:amd64 (2.0.5-5) ...
Selecting previously unselected package python3-li

<ipython-input-1-393135217c75>:6: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


### Mount Google Drive
To stream files we need to mount Google Drive.

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


####**Setup & Configure:**

**Download setup files & configure the environment:**

<b>Options:</b><br>
`INSTALL_RCLONE_FLAVOR`<b>: Switch Rclone's release branch from stable to beta or vice-versa.</b><br>
`FORCE_SETUP`<b>: Forcefully install Rclone by excluding all other dependencies and download them separately. Useful if any dependency (except Rclone) having temporary setup problem.</b><br>
`CUSTOM_SH_FILE`:<b> Get custom .sh file from external source (URL or File Path) and execute it to install additional dependecies or requirements with or without normal setup.</b>

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Setup_img.png" height="50" alt="Setup-Logo"/></center>
#@markdown <center><h3><b>Setup & Configure Environment</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from time import sleep
from requests import get as get_url
from urllib.parse import urlparse as validate_url
from google.colab import output
from json import dumps as dump_json

current_version = "v5.1.0"
do_normal_setup = "1"

# Check for latest version
try:
    json_data = get_url("https://assets.thecaduceus.eu.org/MCT/json/mct.json").json()
    print("Connected with CDN.")
    if current_version == json_data["version"]:
        print(f'Great Job! you are using latest version: {json_data["version"]}')
    else:
        print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]}, use latest notebook available here:\n{json_data["notebookLink"]}')
        sleep(10)
except Exception as error_code:
    end(f'Failed to connect with CDN!\nTraceback:\n{error_code}')

# Save json data locally
with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

# Flavor Flags
Stable = ""
Beta = "-s beta"

# User Input
INSTALL_RCLONE_FLAVOR = Stable #@param ["Stable", "Beta"] {type:"raw"}
FORCE_SETUP = True #@param {type:"boolean"}
CUSTOM_SH_FILE = False #@param {type:"boolean"}

if CUSTOM_SH_FILE:

    Custom_sh_Source = input("Provide the Custom Install.sh file source, it can be URL or File Path:\n")
    use_sudo = input("Do you want to execute the .sh file as sudo? 1 = Yes and 0 = No:\n")

    if use_sudo not in ['0','1']:
        end("Unable to check for latest version!")
    elif not Custom_sh_Source:
        end("Custom sh file source can't be blank!")
    elif all([validate_url(Custom_sh_Source).scheme, validate_url(Custom_sh_Source).netloc]):
        try:
            get_url(Custom_sh_Source)
            if use_sudo == '1':
                !curl {Custom_sh_Source} | sudo bash
                print('Execution completed as sudo!')
            elif use_sudo == '0':
                !curl {Custom_sh_Source} | bash
                print('Execution completed!')
        except ConnectionError:
            end("Unable to connect with given sh file URL.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")
    elif not Path(Custom_sh_Source).exists():
        end("Provided Custom sh file path not exist!")
    elif not Path(Custom_sh_Source).is_file():
        end("Provided Custom sh file path should of a particular file not directory/folder.")
    elif not PurePath(Custom_sh_Source).suffix == '.sh':
        end("File format should be .sh")
    else:
        try:
            if use_sudo == '1':
                !sudo bash "$Custom_sh_Source"
                print("Execution completed as sudo.")
            else:
                !bash "$Custom_sh_Source"
                print("Execution complete.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")


    # Ask for Normal Setup
    do_normal_setup = input('Do you want to run Normal Setup also? 1 = Yes and 0 = No:\n')

    if do_normal_setup not in ['0','1']:
        end("Input Error: Value should be 1 or 0")

# Force Setup
if FORCE_SETUP:
    if Path("/content/setup.sh").exists():
        !rm {"/content/setup.sh"}
    print("Starting Force Setup...")
    sleep(4)
    !curl "https://rclone.org/install.sh" | sudo bash {INSTALL_RCLONE_FLAVOR}

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
    try:
        get_url(json_data["setupFile"])
        !curl {json_data["setupFile"]} | sudo bash {INSTALL_RCLONE_FLAVOR}
        !curl {json_data["setupFile"]} > "/content/setup.sh"
        output.clear()
        print("Setup completed!")
    except ConnectionError:
        print('Unable to get setup file!')
    except Exception as error_code:
        print(f'Setup instructions seems to be having mistake(s)!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
    print("Setup file exist and configured already!")

Setup completed!


####**Rclone Operations:**

In [ ]:
# @markdown <br>
# @markdown <center><h3><b>Connect To Cloud Accounts</b></h3></center>
# @markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Multiple-Clouds.webp" height="100" alt="Multiple-Clouds"/></center>
# @markdown <center><b>Connect your Cloud Accounts (Rclone remotes) between which you want to transfer Data!</b></center>

from sys import exit as end
from os import makedirs, system
from pathlib import Path, PurePath
from re import findall, M

if not Path("/root/.config/rclone/rclone.conf").exists():
  end('Not Found: rclone.conf [file]')

content = open("/root/.config/rclone/rclone.conf").read()
avCon = findall(r"^\[(.+)\]$", content, M)

print("Available Rclone remotes:")
print(*avCon)
remote = input("Enter Rclone remote name to continue:\n")

if remote not in avCon:
  end('Not Found: Given remote name not exist in rclone.conf [file]')

# User Input
CUSTOM_MOUNT_POINT = False  # @param {type:"boolean"}
CUSTOM_CACHE_DIR = False  # @param {type:"boolean"}

# Custom Mount Point
if CUSTOM_MOUNT_POINT:
  Mount_Point = input("Enter Custom Mount path:\n")
  if not Mount_Point:
    end("Mount Path can't be empty.")
  elif Path(Mount_Point).exists() and Path(Mount_Point).is_dir():
      Mount_Point = f"{PurePath(Mount_Point)}/{remote}"
  elif not Path(Mount_Point).exists():
      end('Not Found: Given mount path not found.')
  elif not Path(Mount_Point).is_dir():
    end('Path Error: Given path is not a valid directory.')
else:
  Mount_Point = f"/content/drives/{remote}"

# Custom Cache Directory
if CUSTOM_CACHE_DIR:
  Cache_dir = input("Enter Custom Cache Directory path:\n")
  if not Cache_dir:
    end("Cache Directory Path can't be empty.")
  elif Path(Cache_dir).exists() and Path(Cache_dir).is_dir():
    Cache_dir = f"{PurePath(Cache_dir)}/{remote}"
  elif not Path(Cache_dir).exists():
    end('Not Found: Given mount path not found.')
  elif not Path(Cache_dir).exists():
    end('Path Error: Give path is not a valid directory.')
else:
  Cache_dir = "/content/temp/Rclone"

Action = input("Select Action: 1 = Mount and 0 = Unmount:\n")

if Action not in ['0','1']:
  end("Input Error: Value should be 1 or 0.")
elif Action == '1':
  makedirs(Mount_Point, exist_ok=True)
  !rclone mount {remote}: {Mount_Point} --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir {Cache_dir} --allow-other --daemon
  print(f'Mounted: {Mount_Point}') if Path(Mount_Point).exists() else print(f"Mount Error: Unable to mount {Mount_Point}")
elif Action == '0' and system(f"fusermount -uz {Mount_Point}") == 0:
  !rm -r {Mount_Point}
else:
  system(f"fusermount -uz {Mount_Point}")

**Create/Edit/View/Delete your Rclone Config file:<br>``UPLOAD_CONFIG_FILE`` : Upload your existing Rclone config file. It will be placed in its correct location automatically.<br>``SAVE_TO_MYDRIVE`` : Save Rclone config file in Google Drive.<br>``DOWNLOAD_CONFIG`` : Download Rclone config file in your Device.**


In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center>
#@markdown <center><h3><b>Create/Edit/Delete Rclone Config File</b></h3></center><br>

from google.colab import files
from sys import exit as end
from pathlib import Path, PurePath

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

# Paths
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download Setup first!')
elif not Path(f'{CMP}').exists():
  end('Not Found: Google Drive is not connected!')
elif not Path(DRIVE_CONFIG_DIR).exists():
  !rclone mkdir {DRIVE_CONFIG_DIR}
  print('Created: MCT-Config [folder] -> MyDrive')

#@markdown ****
UPLOAD_CONFIG_FILE = False #@param {type:"boolean"}
SAVE_TO_MYDRIVE = True #@param {type:"boolean"}
DOWNLOAD_CONFIG = True #@param {type:"boolean"}

if UPLOAD_CONFIG_FILE and SAVE_TO_MYDRIVE and DOWNLOAD_CONFIG:
  # Initiate Upload Request
  print('Request: Initiate upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path("/root/.config/rclone/").exists():
    !rclone mkdir "/root/.config/rclone/"
  !rclone copy "/content/rclone.conf" "/root/.config/rclone/"
  print('Saved: rclone.conf [file] -> Runtime')
  # Rclone configuration
  !rclone config
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
  # Initiate Download Request
  files.download("/root/.config/rclone/")
  print('Request: Initiated download -> rclone.conf [file]')
elif UPLOAD_CONFIG_FILE and SAVE_TO_MYDRIVE:
  # Initiate Upload Request
  print('Request: Initiate upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path('/root/.config/rclone/').exists():
    !rclone mkdir "/root/.config/rclone/"
  !rclone copy "/content/rclone.conf" "/root/.config/rclone/"
  print('Saved: rclone.conf [file] -> Runtime')
  # Rclone configuration
  !rclone config
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
elif UPLOAD_CONFIG_FILE:
  # Initiate Upload Request
  print('Request: Initiate Upload -> rclone.conf [file]')
  file = files.upload()
  if list(file.keys())[0] != 'rclone.conf':
    end('File Error: File is not rclone.conf')
  elif not Path('/root/.config/rclone/').exists():
    !rclone mkdir "/root/.config/rclone/"
  print('[WARNING] -> No saving option selected, progress may lose.')
  # Rclone configuration
  !rclone config
elif SAVE_TO_MYDRIVE:
  # Rclone configuration
  !rclone config
  # Save to Google Drive
  !rclone copy "/content/rclone.conf" {DRIVE_CONFIG_DIR}
  print('Saved: rclone.conf [file] -> MyDrive')
elif DOWNLOAD_CONFIG:
  # Rclone configuration
  !rclone config
  # Initiate Download Request
  files.download("/root/.config/rclone/")
  print('Request: Initiated download -> rclone.conf [file]')
else:
  print('[WARNING] -> No saving option selected, progress may lose.')
  !rclone config

**Mount Rclone remotes on this runtime:**

**Options:**<br>
`CUSTOM_MOUNT_POINT`**: Set custom mount point.**<br>
`CUSTOM_CACHE_DIR`**: Set custom cache directory.**

**Using various Rclone modes, flags & other options:**



**Options:**<br>
`DRY_RUN`**: Check given source & destination.**<br>
`EMAIL_NOTIFICATION`**: Receive email notification once task is completed.**<br>
`LOGS`**: Send log file with email notification.**

**Delete your Files/Folder/Directory from below:<br>``delete`` Delete the content of folder or single file.<br>``purge`` Delete whole directory.<br>``rmdir`` Completely delete the given path.<br>``rmdirs`` Remove any empty directories under the path.**

### Add From Torrent File
You can run this cell to add more files as many times as you want

In [3]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

Saving TitanMen collection.torrent to TitanMen collection.torrent


### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [4]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

<ipython-input-4-4fb61c2df196>:30: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
<ipython-input-4-4fb61c2df196>:35: DeprecationWarning: name() is deprecated
  download.name(),


KeyboardInterrupt: 

In [ ]:
# @title 默认标题文本 {"display-mode":"form"}
# @markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/google-cloud-logo.png" height="150" alt="Google-Cloud-Logo"/></center>
# @markdown <center><h3><b>Using Service Accounts</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from google.colab import files

# Where user mounted Google Drive?
try:
    CMP = PurePath(Custom_Mount_Path)
except NameError:
    CMP = '/content/drive'

# Paths
ROOT = "/content/"
DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"
ROOT_ACCOUNTS_ZIP = "/content/accounts.zip"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
    end('Please download Setup first!')
elif not Path(CMP).exists():
    end('Not Found: Google Drive is not connected!')
elif not Path(DRIVE_CONFIG_DIR).exists():
    !rclone mkdir "$DRIVE_CONFIG_DIR"
    print('Created: MCT-Config [folder] -> MyDrive')

# Configure Options
UPLOAD_SERVICE_ACCOUNTS = False  # @param {type:"boolean"}
SAVE_TO_MYDRIVE = True  # @param {type:"boolean"}

if UPLOAD_SERVICE_ACCOUNTS and SAVE_TO_MYDRIVE:
    print('Upload accounts.zip:\n')
    file = files.upload()
    if list(file.keys())[0] == 'accounts.zip':
        !rclone copy "$ROOT_ACCOUNTS_ZIP" "$DRIVE_CONFIG_DIR"
        !unzip "$ROOT_ACCOUNTS_ZIP" -d "$ROOT"
        print('Uploaded & Configured Service Accounts! (2)')
    else:
        end('File Error: File is not accounts.zip')

elif not UPLOAD_SERVICE_ACCOUNTS and not SAVE_TO_MYDRIVE:
    print('Task Error: No Task to perform :)')

elif UPLOAD_SERVICE_ACCOUNTS:
    print('Upload accounts.zip:\n')
    file = files.upload()
    if list(file.keys())[0] == 'accounts.zip':
        !unzip "$ROOT_ACCOUNTS_ZIP" -d "$ROOT"
        print('Uploaded & Configured Service Accounts! (1)')
    else:
        end('File Error: File is not accounts.zip')

elif Path(ROOT_ACCOUNTS_ZIP).exists():
    print('Found: accounts.zip [file]')
    !unzip "$ROOT_ACCOUNTS_ZIP" -d "$ROOT"
    print('Configured Service Accounts!')

else:
    print('Not Found: accounts.zip [file]')